In [178]:
%matplotlib notebook

import os
import sys
from operator import itemgetter
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
from ipyleaflet import (Map, GeoJSON)

sns.set()

from sklearn.cluster import DBSCAN

import matplotlib.dates as mdates
from matplotlib.colors import rgb2hex
import json

def get_geojson(features):
    return {
        'type': 'FeatureCollection',
        'features': features
    }

def save_geojson(features, directory, file_name):
    if not os.path.exists(directory): os.makedirs(directory)
    f = os.path.join(directory, file_name + '.geojson')
    geojson = {
      'type': 'FeatureCollection',
      'features': features
    }
    with open(f, 'w') as outfile:
        json.dump(geojson, outfile, indent = 4)
    print('Saved to ' + f)

def to_geojson(df, groupby, lat, lng, cols, dumps=True):

    def get_features(row, color):
        properties = { k: str(v) for k,v in zip(cols,[row[col] for col in cols]) }
        properties['marker-color'] = rgb2hex(color[:3])
        return {
            'type': 'Feature',
                'geometry': {
                'type': 'Point',
                'coordinates': [row[lng], row[lat]]
            },
            'properties': properties
        }

    clusters = df.groupby(groupby)

    features = []
    colors = plt.cm.Spectral(np.linspace(0, 1, len(clusters)))
    for name, group in clusters:
        i = np.random.randint(colors.shape[0])
        color = colors[i]
        group.apply(lambda row: features.append(get_features(row, color)), axis=1)
        colors = np.delete(colors, i, 0)

    if dumps:
        return json.dumps(get_geojson(features))
    return get_geojson(features)

In [179]:
f = '/Users/Sylvain/Desktop/UTSEUS Data/UTSEUS-shanghai-flickr.sqlite'

In [180]:
import sqlite3
conn = sqlite3.connect(f)
cursor = conn.cursor()
cursor.execute("""
SELECT * FROM photos
""")
photos = pd.DataFrame(cursor.fetchall())

In [181]:
photos.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,12786061,31.239682,121.497266,23804952@N00,15,22726052.0,2004,9,25,Needle in the Sky,sky 2004 architecture shanghai pearltower dscp8,"Pearl TV Tower, Shanghai",http://farm1.staticflickr.com/9/12786061_a6b55...
1,21048909,31.234380,121.494541,40264825@N00,16,22726050.0,2005,3,22,Skywards,holga cityscape shanghai,Holga Shanghai Places,http://farm1.staticflickr.com/16/21048909_3a5c...
2,21048962,31.234380,121.494541,40264825@N00,16,22726050.0,2005,6,23,Bund2,holga cityscape shanghai,Holga Shanghai Places,http://farm1.staticflickr.com/16/21048962_98b0...
3,21048995,31.234380,121.494541,40264825@N00,16,22726050.0,2005,6,23,RoomWithaView,holga cityscape shanghai hotelrooms,Holga Shanghai Places\n\nCamera: Holga 120N\n,http://farm1.staticflickr.com/15/21048995_8cd6...
4,21049047,31.234380,121.494541,40264825@N00,16,22726050.0,2004,11,30,"Bund, Early Morning (Shanghai)",holga cityscape shanghai,Camera: Holga 120N,http://farm1.staticflickr.com/15/21049047_51f8...


In [187]:
# Don't forget to remove "sample" to get full dataset
data_dbscan = photos[[1, 2]].sample(500)
data_dbscan.columns = ['latitude', 'longitude']
data_dbscan.head()
X = data_dbscan.values

In [188]:
kms_per_radian = 6371.0088

def compute_dbscan(meters):
    epsilon = (meters * 0.001) / kms_per_radian
    db = DBSCAN(eps=epsilon, algorithm='ball_tree', metric='haversine').fit(np.radians(X))
    return db.labels_

computings = map(compute_dbscan, range(100, 300))
n_computings = map(lambda x: (len(set(x)), x), computings)
labels = max(n_computings, key=itemgetter(0))[1]

for label in set(labels):
    df = data_dbscan[(labels == label)]
    data_dbscan.loc[df.index, 'cluster_num'] = int(label)

In [189]:
len(set(labels))

28

In [190]:
data_dbscan.sample(n=100)

,latitude,longitude,cluster_num
16067,31.237483,121.499013,0.0
34427,31.188620,121.433193,-1.0
27758,31.240985,121.640625,-1.0
5479,31.221453,121.523733,-1.0
2954,31.211758,121.512407,-1.0
17290,31.236536,121.471516,2.0
29076,31.200000,121.500000,8.0
19254,31.225701,121.445174,26.0
25199,31.238506,121.493583,17.0
19611,31.225083,121.469291,22.0


In [191]:
to_geojson(data_dbscan, 'cluster_num', 'latitude', 'longitude', ['cluster_num'])

'{"features": [{"geometry": {"coordinates": [121.49806, 31.200963], "type": "Point"}, "properties": {"marker-color": "#fcaa5f", "cluster_num": "-1.0"}, "type": "Feature"}, {"geometry": {"coordinates": [121.31378, 31.10513], "type": "Point"}, "properties": {"marker-color": "#fcaa5f", "cluster_num": "-1.0"}, "type": "Feature"}, {"geometry": {"coordinates": [121.685577, 31.190491], "type": "Point"}, "properties": {"marker-color": "#fcaa5f", "cluster_num": "-1.0"}, "type": "Feature"}, {"geometry": {"coordinates": [121.50403, 31.240877], "type": "Point"}, "properties": {"marker-color": "#fcaa5f", "cluster_num": "-1.0"}, "type": "Feature"}, {"geometry": {"coordinates": [121.459623, 31.206045], "type": "Point"}, "properties": {"marker-color": "#fcaa5f", "cluster_num": "-1.0"}, "type": "Feature"}, {"geometry": {"coordinates": [121.513119, 31.232943], "type": "Point"}, "properties": {"marker-color": "#fcaa5f", "cluster_num": "-1.0"}, "type": "Feature"}, {"geometry": {"coordinates": [121.456466,